In [1]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [7]:
import triton_python_backend_utils as pb_utils

In [36]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [37]:
import pandas as pd
import numpy as np
import json

In [38]:
amount = np.array(['$45.23'])

In [39]:
data = pd.DataFrame({'Amount' : amount})
data['Amount'] = data['Amount'].str.slice(1)

In [40]:
data

,Amount
0,45.23


In [41]:
amount = data['Amount'].values

In [42]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [43]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [44]:
model_name = "preprocessing"

In [45]:
amount_data = np.array(["$34.56"])
amount_input = prepare_tensor("AMOUNT", amount_data)
inputs = [amount_input]

In [46]:
outputs = [
    httpclient.InferRequestedOutput("AMOUNT")
]

In [47]:
response = client.infer(model_name,
                        inputs, outputs=outputs)

InferenceServerException: Unable to parse 'datatype': attempt to access JSON non-string as string

In [ ]:
output0_data = response.as_numpy("AMOUNT")

In [ ]:
print(output0_data)

In [34]:
model_config_str = """
name: "preprocessing"
backend: "python"
max_batch_size: 8
input [
    {
        name: "AMOUNT"
        data_type: TYPE_STRING
        dims: [ 1 ]
    }
]
output [
    {
        name: "AMOUNT"
        data_type: TYPE_FP32
        dims: [ 1 ]
    }
]

instance_group [
    {
        count: 1
        kind: KIND_GPU
    }
]
parameters: {
  key: "EXECUTION_ENV_PATH",
  value: {string_value: "$$TRITON_MODEL_DIRECTORY/rapids22.06_cuda11.5_py3.8.tar.gz"}
}
"""

In [35]:
model_config = json.loads(model_config_str)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)